In [ ]:
# https://github.com/aswalin/SQuAD.git
# https://github.com/priya-dwivedi/cs224n-Squad-Project

In [51]:
import urllib.request, json 
import pandas as pd
from pprint import pprint
%jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10


In [60]:

url = "https://raw.githubusercontent.com/aswalin/SQuAD/master/data/dev-v1.1.json"
with urllib.request.urlopen(url) as url:
    data = json.loads(url.read().decode())
    print(data)
    
# with open(filename) as json_file:  
#     data = json.load(json_file)
#     for p in data['people']:
#         print('Name: ' + p['name'])
#         print('Website: ' + p['website'])
#         print('From: ' + p['from'])
#         print('')

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [57]:
valid = pd.read_json(filename)

In [53]:
print(valid.data.iloc[:6])

0    {'title': 'Super_Bowl_50', 'paragraphs': [{'co...
1    {'title': 'Warsaw', 'paragraphs': [{'context':...
2    {'title': 'Normans', 'paragraphs': [{'context'...
3    {'title': 'Nikola_Tesla', 'paragraphs': [{'con...
4    {'title': 'Computational_complexity_theory', '...
5    {'title': 'Teacher', 'paragraphs': [{'context'...
Name: data, dtype: object


In [12]:
dict = valid.iloc[0][0]
dict.keys()

dict_keys(['title', 'paragraphs'])

In [13]:
dict['title']

'Super_Bowl_50'

In [17]:
list = dict['paragraphs']
len(list)

54

In [26]:
dict2 = list[0]
dict2.keys()

dict_keys(['context', 'qas'])

In [27]:
dict2['context']

'Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi\'s Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.'

In [29]:
list = dict2['qas']
len(list)

30

In [55]:
dict3 = list[0]
dict3.keys()

dict_keys(['answers', 'question', 'id'])

In [32]:
dict3['answers']

[{'answer_start': 177, 'text': 'Denver Broncos'},
 {'answer_start': 177, 'text': 'Denver Broncos'},
 {'answer_start': 177, 'text': 'Denver Broncos'}]

In [33]:
dict3['question']

'Which NFL team represented the AFC at Super Bowl 50?'

In [34]:
dict3['id']

'56be4db0acb8001400a502ec'